In [1]:
#Include Libraries:
import pandas as pd
import pickle
import re
import numpy as np

In [2]:
pd.options.mode.chained_assignment = None
with open("../Raw_Data/save_house_phongtro123.pickle","rb") as handle:
    dataset = pickle.load(handle)
dataset = pd.DataFrame(data = dataset)
dataset = dataset.rename(columns = {"area" : "Area"})
dataset.head()

FileNotFoundError: [Errno 2] No such file or directory: '../Raw_Data/save_house_phongtro123.pickle'

In [ ]:
drop_list = []
for index in range(11):
    drop_list.append('Feature' + str(index))
dataset = dataset.drop(columns = drop_list)
dataset.head()

,Location,Price,Name,Area
0,"Địa chỉ: 408/4 Tôn Đức Thắng, Phường Hòa Minh,...",1.3 triệu/tháng,"Cho thuê phòng trọ đường Tôn Đức Thắng, Liên C...",20m2
1,"Địa chỉ: Phường Nại Hiên Đông, Quận Sơn Trà, Đ...",3 triệu/tháng,Nhà biệt thự còn dư nhiều phòng cho thuê lâu d...,25m2
2,"Địa chỉ: k123/97/58 Cù Chính Lan, Phường Hòa K...",1.5 triệu/tháng,"Cho thuê phòng trọ (riêng chủ) gần chợ, gần tr...",15m2
3,"Địa chỉ: Đường Vương Thừa Vũ, Phường An Hải Bắ...",2.6 triệu/tháng,Cho thuê tại Ngã tư đường Vương Thừa Vũ + Trần...,30m2
4,"Địa chỉ: 493 Tôn Đức Thắng, Phường Hòa Khánh N...",3.2 triệu/tháng,Cho thuê phòng trọ full nội thất đẹp. Giá: từ ...,40m2


In [ ]:
#Regex for extracting useful information
price_re = "([(\d\W)*\d+]+(triệu|tỷ|đ)+)?(thoả thuận)?"
price_stn = "(\d+)\W*(\d*) (triệu|đ)"
location_re = "((Phường|Xã)+((\W\w+)+), )*(Quận|Huyện)+((\W\w+)+)"
area_re = "\d+"

#Modifying the dataframe
dataset['Price'] = dataset['Price'].apply(lambda x: re.match(price_re,x)[0])
dataset = dataset[dataset['Price'].str.contains("tỷ") == False]
dataset = dataset.reset_index(drop = True)
dataset['Ward'] = ''
dataset['District'] = ''
for i in range(len(dataset['Location'])):
    try:
        location_components = list(re.findall(location_re,dataset['Location'][i])[0])
        dataset['Ward'][i] = location_components[2]
        dataset['District'][i] = location_components[5]
    except:
        dataset['Ward'][i] = ''
        dataset['District'][i] = ''
dataset['Area'] = dataset['Area'].apply(lambda x: re.findall(area_re, x)[0])
for i in range(len(dataset['Price'])):
    try:
        price_components = list(re.findall(price_stn,dataset['Price'][i])[0])
        if (price_components[2] == 'triệu'):
            if (price_components[1] != ''):
                dataset['Price'][i] = int(float(price_components[0]) * 1000000 + float(str(0.) + price_components[1]) * 10000000)
            else: dataset['Price'][i] = int(float(price_components[0]) * 1000000)
        else:
            dataset['Price'][i] = int(float(''.join([price_components[0],price_components[1]])))
    except: print(dataset['Price'][i])
dataset.head()

,Location,Price,Name,Area,Ward,District
0,"Địa chỉ: 408/4 Tôn Đức Thắng, Phường Hòa Minh,...",1300000,"Cho thuê phòng trọ đường Tôn Đức Thắng, Liên C...",20,Hòa Minh,Liên Chiểu
1,"Địa chỉ: Phường Nại Hiên Đông, Quận Sơn Trà, Đ...",3000000,Nhà biệt thự còn dư nhiều phòng cho thuê lâu d...,25,Nại Hiên Đông,Sơn Trà
2,"Địa chỉ: k123/97/58 Cù Chính Lan, Phường Hòa K...",1500000,"Cho thuê phòng trọ (riêng chủ) gần chợ, gần tr...",15,Hòa Khê,Thanh Khê
3,"Địa chỉ: Đường Vương Thừa Vũ, Phường An Hải Bắ...",2600000,Cho thuê tại Ngã tư đường Vương Thừa Vũ + Trần...,30,An Hải Bắc,Sơn Trà
4,"Địa chỉ: 493 Tôn Đức Thắng, Phường Hòa Khánh N...",3200000,Cho thuê phòng trọ full nội thất đẹp. Giá: từ ...,40,Hòa Khánh Nam,Liên Chiểu


In [ ]:
dataset = dataset.drop(columns = ['Location', 'Name'])
dataset.head()

,Price,Area,Ward,District
0,1300000,20,Hòa Minh,Liên Chiểu
1,3000000,25,Nại Hiên Đông,Sơn Trà
2,1500000,15,Hòa Khê,Thanh Khê
3,2600000,30,An Hải Bắc,Sơn Trà
4,3200000,40,Hòa Khánh Nam,Liên Chiểu


In [ ]:
dataset.to_csv('../Dataset/phongtro123.csv')